# SETUP

In [1]:
import sys
sys.path.append('../')

# RUN

## Prepare data

### Load raw dataset

In [2]:
from src.data import data_provider, data_processor

DATA_ROOT = '/home/victor/Learning/bku/dissertation/implementation/data'
dataset_raw = data_provider.load(
    data_root=DATA_ROOT,
    dataset_name=data_provider.STANDFORD_ONLINE_PRODUCTS,
    dataset_type=data_provider.RAW)

dataset_raw = data_provider.subset(dataset_raw)

Using TensorFlow backend.


Loading dataset: standford_online_products from folder: /home/victor/Learning/bku/dissertation/implementation/data
Try to read from pickle file...
Load data succeed


### Process dataset

In [3]:
# normalize
input_shape = (224, 224, 3)
dataset_inter = data_processor.normalize(dataset_raw, input_shape)

In [4]:
augmentation = {
#     "rotation_range": 20,
#     "horizontal_flip": True
}

dataset_final = data_processor.augment(dataset_inter, augmentation)

Augmenting dataset...
Augmented


## Train CNN

### Load CNN

In [5]:
from src.model import model_provider

model_name = 'vgg16'
num_classes = len(dataset_final['classes'])
cnn_classifier = model_provider.build_cnn_classifier(model_name, input_shape, num_classes)

Model VGG16 has been built
Instructions for updating:
Colocations handled automatically by placer.


### Train CNN

In [6]:
from src import training

# compile model
optimizer_params = {
    "name": "SGD",
    "lr": 0.02,
    "momentum": 0.9
}

cnn_classifier = training.compile_classifier(cnn_classifier, optimizer_params)

In [7]:
# train model
training_params = {
    "batch_size": 32,
    "epochs": 1
}

cnn_classifier, history = training.train_classifier(cnn_classifier, dataset_final, training_params)

Instructions for updating:
Use tf.cast instead.
Train on 100 samples, validate on 20 samples
Epoch 1/1
100/100 [==============================] - 208s 2s/step - loss: 1.1004 - acc: 0.2700 - val_loss: 1.1021 - val_acc: 0.1000
Total training time: 226.6851098537445


In [ ]:
# build cnn_extractor from trained cnn_classifier
cnn_extractor = model_provider.build_cnn_extractor(cnn_classifier, features_layer='fc2')

# evaluate with validation data
from src import experiment
scores = experiment.evaluate_extractor(cnn_extractor, dataset_final, mode='valid', top_k=30)
print('similarity precision:', scores[0])
print('mAP:', scores[1])

### Evaluate CNN